## Merge

In [ ]:
import ROOT
import os

def get_root_files(directory):
    
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.root') and not f.startswith('merge')]

def get_unique_filename(directory, base_name):
    counter = 0
    while True:
        if counter == 0:
            filename = os.path.join(directory, f"{base_name}0.root")
        else:
            filename = os.path.join(directory, f"{base_name}{counter}.root")
        
        if not os.path.exists(filename):
            return filename
        counter = counter + 1

def merge_files(file_list, output_file):
    
    merger = ROOT.TFileMerger()
    merger.OutputFile(output_file)
    
    for i in file_list:
        merger.AddFile(i)
    
    merger.Merge()

directory = 'build/root'
files = get_root_files(directory)
output_filename = get_unique_filename(directory, 'merged')

merge_files(files, output_filename)

## Draw graph

In [ ]:
import ROOT
import os

def plot_xy_scatter_from_root(input_file, tree_name, x_branch, y_branch, output_image):
    
    file = ROOT.TFile(input_file, "READ")
    tree = file.Get(tree_name)
    
    canvas = ROOT.TCanvas("canvas", "X vs Y Scatter Plot", 800, 600)
    graph = ROOT.TGraph()
    
    points = []
    for event in tree:
        x = getattr(event, x_branch)
        y = getattr(event, y_branch)
        points.append((x, y))

    # Sort points based on x-values
    points.sort(key=lambda p: p[0])

    # Create a TGraph and add sorted points
    graph = ROOT.TGraph()
    for i, (x, y) in enumerate(points):
        graph.SetPoint(i, x, y)

    graph.SetTitle("X vs Y Scatter Plot")
    graph.GetXaxis().SetTitle(x_branch)
    graph.GetYaxis().SetTitle(y_branch)
    graph.SetMarkerStyle(20)  # Circular marker
    graph.SetMarkerSize(0.5)
    graph.SetLineColor(ROOT.kBlue)
    graph.SetLineWidth(2)

    graph.Draw("ALP")  # "A" for axes, "P" for points, "L" for line
    
    canvas.SaveAs(output_image)
    file.Close()

directory = 'build/root'
input_file = os.path.join(directory, "merged0.root")
tree_name = "Transportation"
x_branch = "Energy_keV"
y_branch = "Mass_Attenuation"
output_image = "xy_plot.png"

plot_xy_scatter_from_root(input_file, tree_name, x_branch, y_branch, output_image)

## Archivo CSV

In [ ]:
import ROOT
import os
import csv

def copy_data_csv(base_path, csv_file, tree_name, x_branch, y_branch):
    
    with open(csv_file, 'w', newline='') as file:

        writer = csv.writer(file)
        writer.writerow(['File', tree_name, x_branch, y_branch])
        
        for file_name in sorted(os.listdir(base_path)):
            if file_name.endswith('.root') and not file_name.startswith('merge'):

                file_path = os.path.join(base_path, file_name)
                try:

                    root_file = ROOT.TFile(file_path)
                    tree = root_file.Get(tree_name)
                    
                    if tree:
                        branch1 = tree.GetBranch(x_branch)
                        branch2 = tree.GetBranch(y_branch)

                        entries1 = branch1.GetEntries()
                        # entries2 = branch2.GetEntries()

                        for entry in range(entries1):
                            
                            branch1.GetEntry(entry)
                            branch2.GetEntry(entry)
                            value1 = branch1.GetLeaf(x_branch).GetValue()
                            value2 = branch2.GetLeaf(y_branch).GetValue()
                        
                        writer.writerow([file_name, tree_name, value1, value2])
                        
                    root_file.Close()

                except Exception as e:
                    print(f"Error procesando {file_path}: {e}")
                
    print(f"Archivo CSV '{csv_file}' generado exitosamente.")


directory = 'build/root/'
base_path = directory
csv_file = os.path.join(directory, 'Coefficient.csv')
tree_name = "Transportation"
x_branch = "Energy_keV"
y_branch = "Mass_Attenuation"

copy_data_csv(base_path, csv_file, tree_name, x_branch, y_branch)

## Plot CSV

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot(csv_file, tree_name, x_branch, y_branch):

    df = pd.read_csv(csv_file)

    x = df[x_branch]
    y = df[y_branch]

    plt.figure(figsize=(10, 6))
    plt.plot(x, y, marker='o')

    plt.xlabel(x_branch)
    plt.ylabel(y_branch)
    plt.title(tree_name)
    plt.grid(True)
    plt.show()

directory = 'build/root/'
csv_file = os.path.join(directory, 'Coefficient.csv')
tree_name = "Transportation"
x_branch = "Energy_keV"
y_branch = "Mass_Attenuation" 

plot(csv_file, tree_name, x_branch, y_branch)